# Astronomical Image Data Reduction
## for near-infrared wavelengths

Data reduction for near-infrared \(NIR\) images is similar to visible light images except for the added steps of removing the strong atmospheric background. Traditionally this was done using the software package [IRAF](http://iraf.noao.edu/) developed by the National Optical Astronomy Observatory \(NOAO\). This set of tools is a little buggy, and newer generations of astronomers are turning to Python instead.

In this tutorial, we will practice using the [AstroPy](http://docs.astropy.org/en/stable/index.html) package in a Python environment. First load the NumPy, Matplotlib Pyplot, and AstroPy FITS modules



In [2]:
import numpy as np #library for array operations and high-level math functions
import matplotlib.pyplot as plt #for making plots
from astropy.io import fits #to access FITS files

FITS is the standard file format for astronomical imaging data. FITS files contain both multi-dimensional array data (1D spectra, 2D images, 3D+ data cubes) and descriptive information about the data.